In [ ]:
!pip install tensorflow numpy pandas matplotlib scikit-learn


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


2025-02-01 17:57:50.263806: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
!pip install hyperopt

     |████████████████████████████████| 1.6 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 50.4 MB/s eta 0:00:01


In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [2]:
nrows=500000
data = pd.read_csv('/Users/arthur/Documents/STUDY/Imperial/rough paths /salvi notebook/data.csv.gz', compression='gzip', nrows=nrows)


### !! Add a hubertloss criterion for Loss !!

In [11]:


# Load the dataset
final_df = pd.read_csv("/Users/arthur/Documents/STUDY/Imperial/rough paths /salvi notebook/final_dataset_depth-1.csv")

# Split features and target
X = final_df.drop("y", axis=1).values
y = final_df["y"].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for LSTM input
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Hyperparameter tuning using Hyperopt
space = {
    'units': hp.choice('units', [32, 50, 64, 128]),
    'dropout_rate': hp.uniform('dropout_rate', 0.1, 0.5),
    'optimizer': hp.choice('optimizer', ['adam', 'rmsprop']),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01))
}

optimizer_mapping = {0: 'adam', 1: 'rmsprop'}

def build_model(params):
    model = Sequential([
        LSTM(int(params['units']), return_sequences=True, input_shape=(X_train.shape[1], 1)),
        Dropout(params['dropout_rate']),
        LSTM(int(params['units']), return_sequences=False),
        Dropout(params['dropout_rate']),
        Dense(25, activation="relu"),
        Dense(1)
    ])

    optimizer_choice = optimizer_mapping.get(params['optimizer'], 'adam')
    optimizer = Adam(learning_rate=params['learning_rate']) if optimizer_choice == 'adam' else RMSprop(learning_rate=params['learning_rate'])

    model.compile(optimizer=optimizer, loss='mse')
    return model

def objective(params):
    model = build_model(params)
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    val_loss = min(history.history['val_loss'])
    return {'loss': val_loss, 'status': STATUS_OK}

# Optimize hyperparameters
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# Mapping the best optimizer index to the actual optimizer
best_params['optimizer'] = optimizer_mapping.get(best_params['optimizer'], 'adam')

# Final model training with best parameters
final_model = build_model(best_params)
final_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Prediction and evaluation
y_pred = final_model.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/arthur/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



100%|██████████| 50/50 [1:50:08<00:00, 132.16s/trial, best loss: 0.28530532121658325]
Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.3221 - val_loss: 0.3503
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.3157 - val_loss: 0.3499
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.3351 - val_loss: 0.3510
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.3342 - val_loss: 0.3511
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.3408 - val_loss: 0.3479
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.3038 - val_loss: 0.3476
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.3414 - val_loss: 0.3484
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.3191 - val_loss: 0.3470
Epoch 9/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.3197 - val_loss: 0.3476
Epoch 10/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 0.3263 - val_loss: 0.3458
Epoch 11/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s

In [ ]:
# Train the LSTM model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.title("LSTM Training Loss")
plt.legend()
plt.show()
